In [138]:
import networkx as nx
import os
import pandas as pd
import numpy as np
from pandas import DataFrame, Series
import datetime
import pytz
import matplotlib.pyplot as plt
import preprocessing as prep
from collections import namedtuple
from bb_binary import FrameContainer, Repository, load_frame_container
import numpy as np
import numpy.ma as npma
import datetime
import csv
import igraph as ig

In [96]:
%matplotlib inline

In [2]:
Detection = namedtuple('Detection', ['idx', 'xpos', 'ypos', 'radius', 'zRotation', 'decodedId', 'frame_idx', 'timestamp', 'cam_id', 'fc_id'])

In [3]:
def getDF(path, b, e, camID):
    repo = Repository(path)
    
    tpls = []
    myid = 0


    for frame, fc in repo.iter_frames(begin=b, end=e, cam=camID):
        for d in frame.detectionsUnion.detectionsDP:
            d = Detection(d.idx, d.xpos, d.ypos, d.radius, d.zRotation, list(d.decodedId), myid, frame.timestamp, fc.camId, fc.id)
            tpls.append(d)
        myid += 1

    df = DataFrame(tpls)
    return df

In [54]:
start = "2016-07-21T12:03:00Z"
start_dt = datetime.datetime.strptime(start, "%Y-%m-%dT%H:%M:%SZ").replace(tzinfo=pytz.UTC)
start_ts = start_dt.timestamp()
end_dt = start_dt + datetime.timedelta(minutes=45)
end_ts = end_dt.timestamp()

In [55]:
df = getDF("/storage/mi/aschle/hour2016/",start_ts, end_ts, 3)

In [56]:
len(df.frame_idx.unique())

8127

In [11]:
prep.calcIds(df, 0.99, 2015).head()

,idx,xpos,ypos,radius,zRotation,frame_idx,timestamp,cam_id,fc_id,confidence,id
0,0,140,1449,23.079645,-0.250615,0,1.469103e+09,0,12799584830633962843,1.000000,954
2,2,164,1924,22.645014,0.465934,0,1.469103e+09,0,12799584830633962843,1.000000,645
5,5,212,824,22.544197,2.202154,0,1.469103e+09,0,12799584830633962843,0.992157,668
9,9,439,1874,23.073574,-0.601321,0,1.469103e+09,0,12799584830633962843,0.992157,1945
10,10,513,1624,22.943163,-0.637520,0,1.469103e+09,0,12799584830633962843,1.000000,918


In [8]:
prep.calcIds(df, 0.99, 2016).head()

,idx,xpos,ypos,radius,zRotation,frame_idx,timestamp,cam_id,fc_id,confidence,id
3,3,100,3349,22.783443,-1.922754,0,1.469103e+09,1,18337027979960261638,1.000000,1381
6,6,139,1186,22.689045,0.338219,0,1.469103e+09,1,18337027979960261638,1.000000,948
7,7,225,1887,23.002411,-2.043183,0,1.469103e+09,1,18337027979960261638,0.992157,2021
8,8,239,2336,22.770222,-0.047829,0,1.469103e+09,1,18337027979960261638,1.000000,1141
9,9,249,2024,23.074924,-1.874026,0,1.469103e+09,1,18337027979960261638,0.992157,1142


# Look at the network

In [164]:
g = nx.read_graphml("test2016_0.99conf_160dist_6ilen.graphml")

In [165]:
stat = prep.network_statistics(g)

In [166]:
stat

{'av_deg': 9.858123569794051,
 'av_shortest_path': 3.253556347165538,
 'cc': 0.055651926872375926,
 'components': 1,
 'degree': dict_values([15, 10, 20, 11, 7, 1, 18, 2, 13, 1, 6, 9, 4, 15, 30, 15, 23, 1, 7, 11, 3, 6, 7, 30, 3, 10, 10, 6, 10, 6, 3, 3, 3, 21, 6, 5, 3, 9, 5, 6, 5, 3, 3, 1, 3, 13, 19, 17, 2, 6, 3, 38, 2, 11, 1, 1, 2, 2, 3, 1, 5, 17, 11, 10, 9, 3, 5, 1, 2, 4, 14, 6, 6, 26, 12, 1, 21, 11, 1, 35, 25, 3, 16, 13, 2, 16, 7, 28, 1, 4, 6, 1, 11, 13, 17, 10, 15, 2, 14, 8, 2, 1, 3, 8, 10, 4, 4, 21, 19, 19, 6, 15, 7, 16, 6, 1, 5, 7, 19, 1, 18, 4, 2, 14, 4, 2, 13, 42, 1, 16, 4, 1, 18, 10, 19, 7, 4, 5, 18, 23, 9, 18, 22, 35, 11, 4, 17, 13, 9, 17, 4, 14, 15, 9, 16, 17, 9, 8, 1, 23, 3, 3, 22, 12, 11, 1, 3, 7, 21, 3, 15, 1, 15, 11, 7, 18, 29, 7, 13, 3, 5, 31, 21, 1, 8, 33, 10, 4, 18, 3, 9, 2, 7, 1, 3, 5, 6, 7, 11, 17, 20, 1, 10, 8, 4, 13, 2, 11, 6, 23, 6, 5, 9, 11, 12, 5, 2, 1, 3, 5, 14, 16, 10, 20, 15, 1, 3, 5, 10, 2, 4, 8, 23, 14, 20, 4, 36, 7, 12, 3, 4, 5, 14, 7, 1, 5, 1, 10, 16, 4, 2

In [167]:
l = []
for n in g.nodes_iter():
#     if get_hatchdate(int(n)) == None:
#         print(n)
    l.append(get_age(int(n), start_dt))

In [168]:
kcc = list(nx.community.k_clique_communities(g,4))

In [169]:
a = []

In [170]:
for e, group in enumerate(kcc):
    for g in group:
        a.append((e,g))

In [171]:
dfkcc = DataFrame(a, columns=["group", "bee"])

In [172]:
dfkcc["age"] = dfkcc.bee.apply(lambda x: get_age(int(x), start_dt))

In [175]:
dfkcc.groupby(by="group").age.std()

group
0      5.131601
1      0.547723
2      3.403430
3      0.957427
4      4.098780
5      5.066228
6      5.322906
7      5.984106
8      2.516611
9      1.726888
10     6.075909
11     5.500000
12     3.130495
13     3.862210
14     5.656854
15     2.387467
16     3.781534
17     2.500000
18     3.464102
19     0.500000
20     4.000000
21    10.115994
Name: age, dtype: float64

In [174]:
dfkcc.groupby(by="group").size()

group
0     7
1     5
2     4
3     4
4     5
5     4
6     7
7     7
8     4
9     8
10    4
11    4
12    5
13    4
14    4
15    5
16    5
17    4
18    4
19    4
20    4
21    4
dtype: int64

In [88]:
def get_mean_age(ages):
    #  Input: object array (ages of bee group)
    # Output: float (mean)
    if any(age is not None for age in ages):
        return np.mean(ages)
    else:
        return npma.masked_equal(ages, None).mean()
    
def get_std_age(ages):
    #  Input: object array (ages of bee group)
    # Output: float (standard deviation)
    if any(age is not None for age in ages):
        return np.std(ages)
    else:
        ages_edit = []
        for age in ages:
            if age is not None: 
                ages_edit.append(age)
        return np.std(ages_edit)

def get_bees_of_age(ages, age):
    #  Input: numpy object ndarray (age of all bees)
    # Output: numpy object ndarray (bees of specified age)    
    return np.where(ages == age)[0]

def get_all_bees_age(date):
    #  Input: date object
    # Output: numpy object ndarray (age of all bees)
    if isinstance(date, datetime.datetime):
        date = date.date()
    
    ages = []
    with open('hatchdates2016.csv', 'r') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')	
        for row in reader:
            if reader.line_num == 1:
                continue
            if row[2] == '':
                age = None
            else:
                age = (date - str_to_datetime(row[2]).date()).days                
            ages.append(age)
    return np.array(ages)

def get_age(dec12, date):	
    #  Input: integer (12 o'clock, clockwise),
    #		  date object
    # Output: integer (age in days)
    if isinstance(date, datetime.datetime):
        date = date.date()
        
    if get_hatchdate(dec12) is None:
        return None
    return (date - get_hatchdate(dec12)).days

def get_hatchdate(dec12):
    #  Input: integer (12 o'clock, clockwise)
    # Output: date object
    with open('hatchdates2016.csv', 'r') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')		
        for row in reader:
            if dec12 == reader.line_num - 2:
                if (str_to_datetime(row[2]) != None):
                    return str_to_datetime(row[2]).date()
                else:
                    return None
    
def str_to_datetime(string):
    #  Input: string in format '%d.%m.%Y'
    # Output: datetime object
    if string == '':
        return None
    return datetime.datetime.strptime(string, '%d.%m.%Y')
    
def date_to_str(date):
    # Input: date(time) object
    # Output: string
    if date == None:
        return ''
    return date.strftime('%d.%m.%Y')

In [141]:
# Alter der Communities testen

In [142]:
g0 = ig.Graph.Read_GraphML("test2016_0.99conf_160dist_6ilen.graphml")

In [143]:
def getCommunitiesFastGreedy(g):

    fg = g.community_fastgreedy()
    vc = fg.as_clustering(fg.optimal_count)

    membership = vc.membership
    ids = [[] for i in range(len(vc.sizes()))]

    for name, membership in zip(g.vs, membership):
        ids[membership].append(int(name['id']))

    return ids

In [144]:
ccc =getCommunitiesFastGreedy(g0)

In [147]:
[len(x) for x in ccc]

[260, 96, 195, 21, 219, 20, 41, 16, 4, 2]

In [148]:
b = []

In [149]:
for e, group in enumerate(ccc):
    for g in group:
        b.append((e,g))

In [152]:
dfccc = DataFrame(b, columns=["group", "bee"])

In [153]:
dfccc["age"] = dfccc.bee.apply(lambda x: get_age(int(x), start_dt))

In [159]:
dfccc.groupby(by="group").age.mean()

group
0     7.925781
1    13.322917
2     9.973822
3     8.526316
4     8.552995
5    10.200000
6    10.658537
7    14.312500
8     4.500000
9     4.500000
Name: age, dtype: float64

In [155]:
dfccc.groupby(by="group").group.size()

group
0    260
1     96
2    195
3     21
4    219
5     20
6     41
7     16
8      4
9      2
dtype: int64